In [2]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle
import altair as alt
import pytz
import numpy as np

In [252]:
t = datetime.now(pytz.timezone('America/Edmonton')).date() - timedelta(days=8)
t2 = datetime.now(pytz.timezone('America/Edmonton')) - timedelta(days=8)
default_pickle = {
    'accessToken':'',
    'current_data':'',
    'daily_outage_dfs':[0]*7,
    'monthly_outage_dfs':[0]*7,
    'alert_dates':{
        'Biomass & Other':t, 
        'Coal':t,
        'Dual Fuel':t,
        'Energy Storage':t,
        'Hydro':t,
        'Natural Gas':t,
        'Solar':t,
        'Wind':t,
        'Intertie':t
    }
}
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Add default current values to dict
with open('./current_df.pickle', 'rb') as handle:
    current_df = pickle.load(handle)
default_pickle['current_data'] = current_df
# Add default monthly values to dict
with open('./monthly_df.pickle', 'rb') as handle:
    monthly = pickle.load(handle)
for i in range(7):
    default_pickle['monthly_outage_dfs'][i] = (t2,monthly)
# Add default daily values to dict
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
for i in range(7):
    default_pickle['daily_outage_dfs'][i] = (t2,daily)
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [289]:
with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
default_pickle

{'accessToken': '8ssihWHW8D6MrEtSSXwLYjlkK9tNqXnU5Dx43EVy6ncGkW9aQJvOLf7C8ywr7v2pnBRwWc1C6jFnGu8kTOLrWHauUFOodycRyAq6-If3dUP-OhCGVqB0swvZ6av3zEX5XjVfC9a9bkNxC8B2B-HAae8ANBVdIMkp8VerRIxYlxjiAZVgJOmDMMHhvLQ1Uls1jwTbj3UTJf4JopWouDYZbWSaBGlkAWjGLHZ-cZO4Lx5PNmt7St4cxgGUPCo8-Dohu-P8-ypKDUdVt5Rj_vbFAmAjReCyt2pbR2VJT6XvLl8eJPpq24Opu1xnpShxewGjsCFAU73jx1SANOgDNzQ-ogqEXlM',
 'current_data': (datetime.datetime(2022, 9, 12, 15, 45, 37, 978140, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                timeStamp    value    fuelType
  0   2022-09-12 00:00:00  1255.00        Coal
  1   2022-09-12 00:05:00  1258.00        Coal
  2   2022-09-12 00:10:00  1259.00        Coal
  3   2022-09-12 00:15:00  1258.00        Coal
  4   2022-09-12 00:20:00  1258.00        Coal
  ..                  ...      ...         ...
  185 2022-09-12 15:25:00   991.01  Pool Price
  186 2022-09-12 15:30:00   991.01  Pool Price
  187 2022-09-12 15:35:00   991.01  Pool Price
  188 2022-09-12 15:40:00   99

In [284]:
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
print(daily)
# monthly['timeStamp'] = monthly['timeStamp'].dt.tz_localize(tz='America/Edmonton')
# print(monthly)
# with open('./monthly_df.pickle', 'wb') as handle:
#     pickle.dump(monthly, handle, protocol=pickle.HIGHEST_PROTOCOL)

                   timeStamp  value  fuelType
0  2022-09-12 00:00:00-06:00    750  Intertie
1  2022-09-13 00:00:00-06:00      0  Intertie
2  2022-09-14 00:00:00-06:00      0  Intertie
3  2022-09-15 00:00:00-06:00      0  Intertie
4  2022-09-16 00:00:00-06:00      0  Intertie
..                       ...    ...       ...
86 2022-12-07 00:00:00-07:00      0      Wind
87 2022-12-08 00:00:00-07:00      0      Wind
88 2022-12-09 00:00:00-07:00      0      Wind
89 2022-12-10 00:00:00-07:00      0      Wind
90 2022-12-11 00:00:00-07:00      0      Wind

[819 rows x 3 columns]


In [290]:
accessToken = default_pickle['accessToken']
def release_token(accessToken):
    path = '/api/ReleaseToken'
    server = 'api.nrgstream.com'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()
release_token(accessToken)